In [98]:
import re
from collections import Counter

In [99]:
with open('data/first.txt', encoding='utf-8') as file:
    text = file.read()

In [100]:
def text_to_wordlist(sentence):
    regexp = "[^а-яА-Яё]"
    sentence = re.sub(regexp, " ", sentence)
    result = sentence.lower().split()
    return result

In [101]:
def ngrams(word, n=3):
    length = len(word)
    result = [word[i:i+3] for i in range(0, length - 2)]
    return result

In [102]:
def make_dict(source_text, accuracy = 0., n=3):
    source_text = ["#"*(n - 1) + word + "#"*(n - 1) for word in text_to_wordlist(source_text)]
    all_ngrams = [ngram for word in source_text for ngram in ngrams(word)]
    counter = Counter(all_ngrams)
    amount_ngrams = len(all_ngrams)
    result_dict = dict()
    for item, count in sorted(counter.items(), key=lambda item: item[1], reverse=True):
        if count/amount_ngrams >= accuracy:
            result_dict[item] = count/amount_ngrams
    return result_dict

In [103]:
result_dict = make_dict(text)
print (result_dict)

{'о##': 0.018347568131002065, 'и##': 0.015263324800929153, 'а##': 0.014254422644367342, 'е##': 0.013482358261958635, '##с': 0.013234815955640418, '##п': 0.01312509450094802, '##в': 0.012878890261150442, '##н': 0.01265275701916245, '##о': 0.010367339401911027, 'я##': 0.0095123148952227, '##к': 0.009469496766562252, '##и': 0.009144346602046974, 'ь##': 0.008860676499671505, 'л##': 0.006749207530103152, 'й##': 0.006585963414585193, '#и#': 0.00657659694894072, '##б': 0.0063129978443748356, 'м##': 0.00629292684656525, '##т': 0.006291588780044611, '#по': 0.006167148593625184, '##д': 0.00590221142253866, 'у##': 0.005583751590626577, '#не': 0.005274658224358966, 'в##': 0.005237192361781074, '##м': 0.004910704130745156, 'то#': 0.004854505336878317, '#на': 0.004451747314165976, '##г': 0.004438366648959587, '##ч': 0.004391534320737221, 'на#': 0.004347378125556134, 'го#': 0.004240332803905013, 'к##': 0.004220261806095429, '#пр': 0.004003495029751909, 'т##': 0.003916520705910373, 'н##': 0.0038375747

In [81]:
def check_word(word, result_dict, accuracy, n = 3):
    word = "#"*(n - 1) + word + "#"*(n - 1)
    text_ngrams = ngrams(word, n)
    flag = True
    for item in text_ngrams:
        if item not in result_dict or result_dict[item] < accuracy:
            print('miss')
            flag = False
            break
    if flag:
        print('exist')

In [85]:
check_word('###', result_dict, 0)

miss


In [88]:
check_word('питание', result_dict, 0.0001)

exist


In [89]:
check_word('пита', result_dict, 0.00001)

exist


In [123]:
class Node:
    def __init__(self, _letter):
        self.children = {}
        self.letter = _letter
        self.end = False
    def add_word(self, word):
        if word[0] in self.children.keys():
            if len(word) > 1:
                self.children[word[0]].add_word(word[1:])
        else:
            next_node = Node(word[0])
            self.children[word[0]] = next_node
            if len(word) > 1:
                next_node.add_word(word[1:])
            elif len(word) == 1:
                next_node.end = True

In [124]:
source_text = [word.lower() for word in text_to_wordlist(text)]

root = Node("~")
for word in source_text:
    root.add_word(word)

In [127]:
root.children['к'].children['н'].children['я'].children['з'].children['ь'].end

True

In [141]:
def find_close(root, word, accuraccy=2):
    word = word.lower()
    possible_answers = [(root, "", 0)]
    for letter in word:
        new_possible_answers = []
        for node, word, errors in possible_answers:
            for key, value in node.children.items():
                if letter == key:
                    new_possible_answers.append((value, word + key, errors))
                else:
                    if errors < accuraccy:
                        new_possible_answers.append((value, word + key, errors + 1))
        possible_answers = [answer for answer in possible_answers if answer[2] + len(word) - len(answer[1]) <= accuraccy 
                           and len(answer[0].children) == 0]
        possible_answers += new_possible_answers
    possible_answers = [answer for answer in possible_answers if answer[2] + len(word) - len(answer[1]) <= accuraccy 
                           and len(answer[0].children) == 0]
    for answer in possible_answers:
        print("Слово - {}, расстояние - {}".format(answer[1], answer[2]))
        
find_close(root, "военныйh")

Слово - верный, расстояние - 1
Слово - вечный, расстояние - 1
Слово - военную, расстояние - 2
Слово - военной, расстояние - 1
Слово - военное, расстояние - 2
Слово - военные, расстояние - 1
Слово - военных, расстояние - 1
Слово - военный, расстояние - 0
Слово - пленный, расстояние - 2
Слово - военную, расстояние - 2
Слово - военной, расстояние - 2
Слово - военное, расстояние - 2
Слово - военные, расстояние - 1
Слово - военных, расстояние - 1
Слово - военный, расстояние - 1
Слово - военную, расстояние - 2
Слово - военной, расстояние - 2
Слово - военное, расстояние - 2
Слово - военные, расстояние - 2
Слово - военных, расстояние - 2
Слово - военный, расстояние - 2
Слово - военными, расстояние - 2
